# Preprocesamiento de Datos de Mortalidad por Asma (parte 1)

En la parte 1 del preprocesamiento de datos haremos la limpieza de datos de mortalidad por asma. El procesamiento consistirá en: i) Manejo de datos faltantes; ii) Remoción de duplicados; y iii) Validación de datos

## Cargar librerías

Se cargarán las librerías necesarias para el análisis

In [68]:
# DataFrame libraries
import pandas as pd
from pandas.api.types import CategoricalDtype

# other libraries
import csv


## Cargar de datos

Se cargarán y vizualizarán los datos suministrados para su preprocesamiento y análisis

In [141]:
# open xls file
df = pd.read_excel("pdt/mortalidad_asma/data/xls/def_asma_2001_2023.xlsx")

In [142]:
# visualizar dataframe
df.head()

,Año,Prov_resid,Dep_Resid,Sexo,Grupedad,Cantidad
0,2001,2,999,2,03.45 a 64,1
1,2001,2,999,2,05.75 y más,1
2,2001,2,999,1,04.65 a 74,1
3,2001,2,999,1,04.65 a 74,1
4,2001,2,999,2,05.75 y más,1


In [143]:
# length of dataframe
len(df)

8404

In [144]:
# excluir datos del 2023
df = df[df['Año'] != 2023]

In [145]:
# valores unicos de Año
df["Año"].unique()

array([2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011,
       2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022])

In [146]:
# length of dataframe
len(df)

8178

In [147]:
# difference in length of dataframe
8404-8178

226

Se excluyeron **226 (8404-8178) registros** correpondientes al año 2023

## Valores faltantes

Se identificarán valores faltantes en cada columna



In [148]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 8178 entries, 0 to 8177
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Año         8178 non-null   int64 
 1   Prov_resid  8178 non-null   int64 
 2   Dep_Resid   8178 non-null   int64 
 3   Sexo        8178 non-null   int64 
 4   Grupedad    8178 non-null   object
 5   Cantidad    8178 non-null   int64 
dtypes: int64(5), object(1)
memory usage: 447.2+ KB


## Valores duplicados
Para este dataset no es necesario remover valores dulicados en  Año + Prov_resid+ Dep_Resid + Sexo + Grupedad

## Validar datos
La validación de datos consiste en verificar y modificar datos de manera que sus valores sean consistentes con sus clases correspondientes. Se efectuarán las siguientes opraciones:

* Generar una columna de ID de departamento (IDDPTO) de manera que coincida con las divisiones administrativas oficiales de Argentina.

* Verificar los valores de sexo (p. ej., "Masculino", "Femenino", "Otro/Desconocido").

* Verificar que las categorías de grupo de edad sean consistentes (p. ej., "0-4", "5-14", "15-24", etc.).

* Asegúrarse de que el número de muertes no sea negativo.





### Generar una columna de ID de departamento (IDDPTO)



Se elimirán muestras con **provicias y departamentos** no definidos

In [149]:
# delete 98 (otros países) and 99 (sin especificar) for 'Prov_resid'
df = df[(df['Prov_resid'] != 98) & (df['Prov_resid'] != 99)]

In [150]:
# length of dataframe
len(df)

8155

In [151]:
# difference in length of dataframe
8178-8155

23

Se eliminaron **23 (8178-8155) registros** pertecientes a otros paises y a provincias sin especificar

Para asegúrese de que la identificación del departamento coincida con las divisiones administrativas oficiales en Argentina, se crearán dos funciones para: 1) añadir 0 a los dígitos menores a 10  en la columna "Prov_resid"; 2) añadir 0 a los dígitos menores a 100 en la columna "Dep_Resid".

In [152]:
# for df['Prov_resid'] append 0 to digts  less than 10

# Convert 'Prov_resid' to string type to allow string operations
df['Prov_resid'] = df['Prov_resid'].astype(str)

# Function to append '0' if the number is less than 10
def append_zero(prov):
    if len(prov) < 2:  # Check if the string length is less than 2.
      return "0" + prov
    return prov

In [153]:
# Apply the function to the 'PROVRE' column
df['Prov_resid'] = df['Prov_resid'].apply(append_zero)

In [154]:
# valores unicos 'Prov_resid'
df['Prov_resid'].unique()

array(['02', '06', '10', '14', '18', '22', '30', '34', '38', '42', '46',
       '50', '54', '58', '62', '66', '74', '82', '86', '90', '94', '26',
       '70', '78'], dtype=object)

La DEIS consideró a la CABA como una unidad geográfica en los datos suministrados. La subdivisión de la CABA no fue homogénea a lo largo del período solicitado (2001-2022). Por lo tanto, se tratará a la CABA como un solo departamento, asignando el valor '000' a todos sus polígonos en la columna 'Dep_Resid'

In [155]:
# for Dep_Resid == 02 assign  000 for IDDPTO

# Convert 'Dep_Resid' to string type if it's not already
df['Prov_resid'] = df['Prov_resid'].astype(str)

# Create a boolean mask where Dep_Resid is '02'
mask = df['Prov_resid'] == '02'

# Assign '02000' to Dep_Resid where the mask is True
df.loc[mask, 'Dep_Resid'] = '000'

/tmp/ipykernel_9/319153812.py:10: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '000' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.loc[mask, 'Dep_Resid'] = '000'


In [156]:
# vizualizar df
df.head()

,Año,Prov_resid,Dep_Resid,Sexo,Grupedad,Cantidad
0,2001,02,000,2,03.45 a 64,1
1,2001,02,000,2,05.75 y más,1
2,2001,02,000,1,04.65 a 74,1
3,2001,02,000,1,04.65 a 74,1
4,2001,02,000,2,05.75 y más,1


In [157]:
# valores uncos 'Dep_Resid'
df['Dep_Resid'].unique()

array(['000', 28, 35, 91, 126, 196, 203, 217, 252, 260, 270, 274, 322,
       357, 371, 410, 412, 413, 427, 441, 462, 505, 539, 568, 595, 638,
       644, 658, 707, 714, 760, 833, 840, 999, 49, 14, 21, 42, 63, 84, 98,
       105, 112, 119, 140, 147, 7, 70, 77, 161, 15, 113, 56, 210, 238,
       408, 420, 434, 455, 490, 511, 515, 532, 560, 581, 623, 651, 749,
       756, 805, 861, 133, 294, 497, 648, 665, 672, 770, 826, 182, 8, 175,
       189, 336, 525, 700, 728, 735, 778, 791, 847, 854, 134, 231, 266,
       277, 655, 763, 798, 43, 385, 469, 609, 882, 88, 364, 588, 693, 819,
       154, 168, 36, 476, 686, 875, 301, 280, 483, 784, 547, 630, 245,
       315, 518, 39, 287, 392, 574, 351, 868, 679, 812, 224, 616, 721,
       602, 742, 406, 466, 399, 218, 448, 94, 329, 343, 553], dtype=object)

In [158]:
# select all samples where Dep_Resid == 999
selected_samples = df[df['Dep_Resid'] == 999]
selected_samples

,Año,Prov_resid,Dep_Resid,Sexo,Grupedad,Cantidad
50,2001,06,999,2,04.65 a 74,1
314,2001,18,999,2,04.65 a 74,1
350,2001,62,999,2,03.45 a 64,1
483,2001,66,999,2,03.45 a 64,1
484,2001,66,999,2,04.65 a 74,1
...,...,...,...,...,...,...
7254,2019,30,999,2,03.45 a 64,1
7319,2019,86,999,2,03.45 a 64,1
7357,2020,06,999,1,04.65 a 74,1
7928,2022,78,999,2,04.65 a 74,1


In [159]:
# delete 999 (sin especificar) for 'Dep_Resid'
df = df[(df['Dep_Resid'] != 999)]

In [160]:
# valores unicos 'Dep_Resid'
df['Dep_Resid'].unique()

array(['000', 28, 35, 91, 126, 196, 203, 217, 252, 260, 270, 274, 322,
       357, 371, 410, 412, 413, 427, 441, 462, 505, 539, 568, 595, 638,
       644, 658, 707, 714, 760, 833, 840, 49, 14, 21, 42, 63, 84, 98, 105,
       112, 119, 140, 147, 7, 70, 77, 161, 15, 113, 56, 210, 238, 408,
       420, 434, 455, 490, 511, 515, 532, 560, 581, 623, 651, 749, 756,
       805, 861, 133, 294, 497, 648, 665, 672, 770, 826, 182, 8, 175, 189,
       336, 525, 700, 728, 735, 778, 791, 847, 854, 134, 231, 266, 277,
       655, 763, 798, 43, 385, 469, 609, 882, 88, 364, 588, 693, 819, 154,
       168, 36, 476, 686, 875, 301, 280, 483, 784, 547, 630, 245, 315,
       518, 39, 287, 392, 574, 351, 868, 679, 812, 224, 616, 721, 602,
       742, 406, 466, 399, 218, 448, 94, 329, 343, 553], dtype=object)

In [161]:
# tamaño de df
len(df)

8070

In [162]:
# diferencia en el tamaño de df
8155-8070

85

Se eliminaron **85 (8155-8070) registros** pertecientes a departamentos sin especificar

In [163]:
# for df['Dep_Resid'] append 0 to digts  less than 100

# Convert 'Dep_Resid' to string type to allow string operations
df['Dep_Resid'] = df['Dep_Resid'].astype(str)

# Function to append '0' if the number is less than 100
def append_zero(prov):
    if len(prov) < 2:  # Check if the string length is less than 2.
      return "00" + prov
    else:
      if len(prov) < 3:
        return "0" + prov
    return prov

In [164]:
# Apply the function to the 'PROVRE' column
df['Dep_Resid'] = df['Dep_Resid'].apply(append_zero)

In [165]:
# valores unicos 'Dep_Resid'
df['Dep_Resid'].unique()

array(['000', '028', '035', '091', '126', '196', '203', '217', '252',
       '260', '270', '274', '322', '357', '371', '410', '412', '413',
       '427', '441', '462', '505', '539', '568', '595', '638', '644',
       '658', '707', '714', '760', '833', '840', '049', '014', '021',
       '042', '063', '084', '098', '105', '112', '119', '140', '147',
       '007', '070', '077', '161', '015', '113', '056', '210', '238',
       '408', '420', '434', '455', '490', '511', '515', '532', '560',
       '581', '623', '651', '749', '756', '805', '861', '133', '294',
       '497', '648', '665', '672', '770', '826', '182', '008', '175',
       '189', '336', '525', '700', '728', '735', '778', '791', '847',
       '854', '134', '231', '266', '277', '655', '763', '798', '043',
       '385', '469', '609', '882', '088', '364', '588', '693', '819',
       '154', '168', '036', '476', '686', '875', '301', '280', '483',
       '784', '547', '630', '245', '315', '518', '039', '287', '392',
       '574', '351',

Crear **identificación única de departamneto (IDDPTO)** con base a "Prov_resid" + "Dep_Resid" de acuerdo a los códigos geográficos del INDEC (https://www.indec.gob.ar/indec/web/Nivel3-Tema-1-39).

In [166]:
# concat Prov_resid	 + Dep_Resid in IDDPTO
df['IDDPTO'] = df['Prov_resid'] + df['Dep_Resid']

In [167]:
# IDDPTO to string
df['IDDPTO'] = df['IDDPTO'].astype(str)

In [168]:
# visualizar df
df.head()

,Año,Prov_resid,Dep_Resid,Sexo,Grupedad,Cantidad,IDDPTO
0,2001,02,000,2,03.45 a 64,1,02000
1,2001,02,000,2,05.75 y más,1,02000
2,2001,02,000,1,04.65 a 74,1,02000
3,2001,02,000,1,04.65 a 74,1,02000
4,2001,02,000,2,05.75 y más,1,02000


In [169]:
# renombrar columnas
df = df.rename(columns={'Año': 'ANIO', "Sexo": "SEXO", "Grupedad": "GRUPEDAD", "Cantidad": "CANTIDAD"})

In [170]:
# seleccionar ANIO, IDDPTO, SEXO, GRUPEDAD, CANTIDAD
df = df[['ANIO', 'IDDPTO', 'SEXO', 'GRUPEDAD', 'CANTIDAD']]

In [171]:
# visualizar df
df.head()

,ANIO,IDDPTO,SEXO,GRUPEDAD,CANTIDAD
0,2001,02000,2,03.45 a 64,1
1,2001,02000,2,05.75 y más,1
2,2001,02000,1,04.65 a 74,1
3,2001,02000,1,04.65 a 74,1
4,2001,02000,2,05.75 y más,1


In [172]:
# save df as csv
df.to_csv("pdt/mortalidad_asma/data/csv/def_asma_2001_2022_clean_01.csv", index=False)

### Verificar los valores de sexo

In [173]:
# open dataframe
df = pd.read_csv("pdt/mortalidad_asma/data/csv/def_asma_2001_2022_clean_01.csv")

In [174]:
# visualizar df
df.head()

,ANIO,IDDPTO,SEXO,GRUPEDAD,CANTIDAD
0,2001,2000,2,03.45 a 64,1
1,2001,2000,2,05.75 y más,1
2,2001,2000,1,04.65 a 74,1
3,2001,2000,1,04.65 a 74,1
4,2001,2000,2,05.75 y más,1


In [175]:
# tamaño del df
len(df)

8070

In [176]:
# valores unicos "SEXO"
df['SEXO'].unique()

array([2, 1, 9])

In [177]:
# delete samples where df['SEXO']==9
df = df[df['SEXO'] != 9]

In [178]:
# valores unicos "SEXO"
df['SEXO'].unique()

array([2, 1])

In [179]:
# tamaño del df
len(df)

8057

In [180]:
# diferencia en el tamaño del df
8070-8057

13

Se eliminaron 13 regitros (8070-8057) con sexo no definido

In [181]:
# Convert 'SEXO' to categorical data
df['SEXO'] = pd.Categorical(df['SEXO'])


In [183]:
# valores unicos ['SEXO'
df['SEXO'].unique()

[2, 1]
Categories (2, int64): [1, 2]

### Verificar las categorías de grupo de edad

In [184]:
# valores unicos 'GRUPEDAD'
df['GRUPEDAD'].unique()

array(['03.45 a 64', '05.75 y más', '04.65 a 74', '02.20 a 44',
       '01.5 a 19', '00.0 a 4', '10.Sin esp'], dtype=object)

In [185]:
# eliminar 'GRUPEDAD'== '10.Sin esp'
df = df[df['GRUPEDAD'] != '10.Sin esp']

In [186]:
len(df)

8028

In [187]:
# diferencia tamaño df
8057-8028

29

Se eliminaron 29 registros (8057-8028) con GRUPEDAD sin especificar

Renombrar valores en  grupoedad


In [188]:
# change values for df['GRUPEDAD']
df['GRUPEDAD'] = df['GRUPEDAD'].replace('00.0 a 4', '0-4')
df['GRUPEDAD'] = df['GRUPEDAD'].replace('01.5 a 19', '5-19')
df['GRUPEDAD'] = df['GRUPEDAD'].replace('02.20 a 44', '20-44')
df['GRUPEDAD'] = df['GRUPEDAD'].replace('03.45 a 64', '45-64')
df['GRUPEDAD'] = df['GRUPEDAD'].replace('04.65 a 74', '65-74')
df['GRUPEDAD'] = df['GRUPEDAD'].replace('05.75 y más', '>= 75')

In [189]:
# valores unicos 'GRUPEDAD'
df['GRUPEDAD'].unique()

array(['45-64', '>= 75', '65-74', '20-44', '5-19', '0-4'], dtype=object)

In [190]:
# Convert 'GRUPEDAD' to ordinal data

# Define the desired order of categories
categories = ['0-4', '5-19', '20-44', '45-64', '65-74', '>= 75']

# Create a categorical data type with the specified order
cat_type = CategoricalDtype(categories=categories, ordered=True)

# Convert the 'GRUPEDAD' column to the ordinal categorical type
df['GRUPEDAD'] = df['GRUPEDAD'].astype(cat_type)


In [191]:
# valores unicos 'GRUPEDAD'
df['GRUPEDAD'].unique()

['45-64', '>= 75', '65-74', '20-44', '5-19', '0-4']
Categories (6, object): ['0-4' < '5-19' < '20-44' < '45-64' < '65-74' < '>= 75']

### Asegúrarse de que el número de muertes no sea negativo.

In [193]:
 # valores unicos 'CANTIDAD'
df['CANTIDAD'].unique()

array([ 1,  2,  3,  5,  4,  7, 13,  6,  8, 10,  9, 11, 12, 16, 14, 15])

In [194]:
# verificar valores nulos
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 8028 entries, 0 to 8069
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype   
---  ------    --------------  -----   
 0   ANIO      8028 non-null   int64   
 1   IDDPTO    8028 non-null   int64   
 2   SEXO      8028 non-null   category
 3   GRUPEDAD  8028 non-null   category
 4   CANTIDAD  8028 non-null   int64   
dtypes: category(2), int64(3)
memory usage: 266.9 KB


In [195]:
# save df as csv
df.to_csv("pdt/mortalidad_asma/data/csv/def_asma_2001_2022_clean_02.csv", index=False)